In [30]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
import numpy as np
# import acquire1
import re
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from env import github_token, github_username

From reading the script documentation it seems that the acquire is taken care of and pulls in a JSON data file that hold all of the data.

From here I will need to import the results and see what I can do with them

In reviewing the material inside of the JSON file I am going to need to declutter the script.
- There are many free floating characters present 
- Turn this clean data into a pandas dataframe

<a href="https://ds.codeup.com/nlp/explore/">This chapter</a>

In [ ]:
df = pd.read_json("data.json")
df

In [ ]:
df.language.value_counts()

Restarted approach to getting all of the text formatted

In [ ]:
df.readme_contents

In [ ]:
type(df.readme_contents)

Error code above indicates that re.sub can only take in string or byte-like objects:

Can run through a for loop with the chnages being applied

In [ ]:
nltk.download('words')
words = set(nltk.corpus.words.words())

In [ ]:
def show_all_matches(regexes, subject, re_length=6):
    print('Sentence:')
    print()
    print('    {}'.format(subject))
    print()
    print(' regexp{} | matches'.format(' ' * (re_length - 6)))
    print(' ------{} | -------'.format(' ' * (re_length - 6)))
    for regexp in regexes:
        fmt = ' {:<%d} | {!r}' % re_length
        matches = re.findall(regexp, subject)
        if len(matches) > 8:
            matches = matches[:8] + ['...']
        print(fmt.format(regexp, matches))


So from here I can find specifically the https website code,

that would be semantics some of these misspelled words could clue us in on where to go next

In [ ]:
def prepare_blogs(Series):
    """ This function takes in a Series and applies a series of cleaning functions before stemming and lemmatizing the text.
    Args: Series
    Function: .lower, normalize, remove non-ASCII, stem, lem
    Returns: Cleaned, stemmed, lemmatized Series in a DF, along with original text"""
    original_content = []
    clean_content = []
    stemmed_content = []
    lemmed_content = []
    blogs_dict = {'content': original_content,
    'cleaned_content': clean_content,
    'stemmed_content': stemmed_content,
    'lemmed_content': lemmed_content}
    for i in range(0, len(Series)):
        content = Series[i]
        # add unaltered text to list 'original_content'
        original_content.append(content)
        # convert to lower case
        content = content.lower()
        # remove accented characters
        # unicode: removes character encoding incosistencies
        # .encode: converts resulting str chars to ASCII set. ignore errors will drop no ASCII chars
        # .decode turns the bytes object back into an str
        content = unicodedata.normalize('NFKD', content)\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
        # remove non-standard alphanumeric characters
        content = re.sub(r"[^a-z0-9'\s]", '', content)
        # tokenization:
        # break words and punctuation left over into discrete units
        tokenizer = nltk.tokenize.ToktokTokenizer()
        tokenizer.tokenize(content, return_str=True)
        # add the tokenized text to the list 'clean_content'
        clean_content.append(content)
        # stems are the base of words, call: calls, called, calling
        # to stem, create the object first:
        ps = nltk.porter.PorterStemmer()
        # then apply to all words in the article
        stems = [ps.stem(word) for word in content.split()]
        content_stemmed = ' '.join(stems)
        #add stemmed output to list:
        stemmed_content.append(content_stemmed)
        # lemmatizing: reduces the word by removing the suffix (if applicable), but leaves a lexi cor word
        # how to lemmatize:
        wnl = nltk.stem.WordNetLemmatizer()
        lemmas = [wnl.lemmatize(word) for word in content.split()]
        content_lemmatized = ' '.join(lemmas)
        # add lemmed content to list:
        lemmed_content.append(content_lemmatized)
    df = pd.DataFrame(blogs_dict)
    return df

In [ ]:
cleaned_df = prepare_blogs(df.readme_contents)

In [ ]:
cleaned_df

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df.lemmed_content[0].split()

In [ ]:
def splitwords(Series):
        terms_list= []
        frequencey_list = []
        augmented_frequency_list = []
        split_word_dict = {'terms_list': terms_list,
        'frequencey_list': frequencey_list,
        'augmented_frequency_list': augmented_frequency_list,}
        for i in range(0, len(Series)):
                word_list = Series[i]
                for word in word_list:
                        term = word 
                        terms_list.append(term)
                        frequency = word_list.value_counts() / word_list.value_counts().sum()
                        frequencey_list.append(frequency)
                        augmented_frequency = frequency / frequency.max()
                        augmented_frequency_list.append(augmented_frequency)
        return pd.DataFrame(split_word_dict)

In [ ]:
new_frame = cleaned_df.lemmed_content.to_frame()
new_frame

s = pd.Series(['A', 'B', 'C'])

for index, value in s.items():

    print(f"Index : {index}, Value : {value}")
Index : 0, Value : A

Index : 1, Value : B

Index : 2, Value : C

In [ ]:
new_frame.lemmed_content.apply(lambda r: r.split())

In [ ]:
ee = new_frame.lemmed_content.apply(lambda r: r.split())

In [ ]:
type(ee)

In [ ]:
type(ee[0])

In [ ]:
ee

In [ ]:
# document = 'Mary had a little lamb, a little lamb, a little lamb.'

# # clean up the text
# document = document.lower().replace(',', '').replace('.', '')
# # transform into a series
# words = pd.Series(document.split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': new_frame.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))


In [ ]:
#the solution will come for the apply method
#the problem pointed out with ee is that the string are in fact split but they return one string that has all these entries
#now then to make the most of the these lists
#I would need to make each word its own index point in a series 
#the problem here is that if I end  up splitting it this way I lose the index and we lose where these words come from
#this can
words
#words in this specific scenario are a pandas series

word_counts = (pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)
                .set_axis(['all', 'ham', 'spam'], axis=1, inplace=False)
                .fillna(0)
                .apply(lambda s: s.astype(int)))

word_counts.head()
i can label spam and ham as i need to for programming languages 
the all count will tell me all of the instances of where it came

In [ ]:
new_frame

In [ ]:
# for i in new_frame.lemmed_content:
word_list = new_frame.lemmed_content[0].split()

In [ ]:
type(word_list)

In [ ]:
words_list = pd.Series(new_frame.lemmed_content[0].split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': words_list.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))


In [ ]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels
#same information as value_counts()

In [ ]:
df.language.value_counts()

In [31]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]


In [105]:
JavaScript = clean(' '.join(df[df.language == 'JavaScript'].readme_contents))
Java = clean(' '.join(df[df.language == 'Java'].readme_contents))
Cplusplus = clean(' '.join(df[df.language == 'C++'].readme_contents))
C_sharp = clean(' '.join(df[df.language == 'C#'].readme_contents))
TypeScript = clean(' '.join(df[df.language == 'TypeScript'].readme_contents))
Ruby = clean(' '.join(df[df.language == 'Ruby'].readme_contents))
Cee = clean(' '.join(df[df.language == 'C'].readme_contents))
PeeHpee = clean(' '.join(df[df.language == 'PHP'].readme_contents))
Jupyter_Notebook = clean(' '.join(df[df.language == 'Jupyter Notebook'].readme_contents))
Assembly = clean(' '.join(df[df.language == 'Assembly'].readme_contents))

all_words = clean(' '.join(df.readme_contents))


In [106]:
JavaScript_freq = pd.Series(JavaScript).value_counts()
Java_ferq = pd.Series(Java).value_counts()
Cplusplus_freq = pd.Series(Cplusplus).value_counts()
C_sharp_freq = pd.Series(C_sharp).value_counts()
TypeScript_freq = pd.Series(TypeScript).value_counts()
Ruby_freq = pd.Series(Ruby).value_counts()
Cee_freq = pd.Series(Cee).value_counts()
PeeHpee_freq = pd.Series(PeeHpee).value_counts()
Jupyter_Notebook_freq = pd.Series(Jupyter_Notebook).value_counts()
Assembly_freq = pd.Series(Assembly).value_counts()
all_freq = pd.Series(all_words).value_counts()

# all_freq.head(20)
# creating a wordcloud from here will yeild a visualization

In [107]:
word_counts = (pd.concat([all_freq, JavaScript_freq, Java_ferq, Cplusplus_freq, C_sharp_freq, TypeScript_freq, Ruby_freq, Cee_freq, PeeHpee_freq, Jupyter_Notebook_freq, Assembly_freq], axis=1, sort=True)
                .set_axis(['all_words', 'JavaScript', 'Java', 'Cplusplus', 'C_sharp', 'TypeScript', 'Ruby', 'Cee', 'PeHPe', 'Jupyter_Notebook', 'Assembly'], axis=1, inplace=False)
                .fillna(0)
                .apply(lambda s: s.astype(int)))

word_counts.head()

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
0,229,79,7,23,7,3,2,1,3,28,0
00,7,0,0,0,0,0,0,0,0,7,0
000,2,0,0,0,0,1,0,0,0,0,0
0000,1,1,0,0,0,0,0,0,0,0,0
000000,2,0,0,0,1,1,0,0,0,0,0


In [108]:
word_counts

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
0,229,79,7,23,7,3,2,1,3,28,0
00,7,0,0,0,0,0,0,0,0,7,0
000,2,0,0,0,0,1,0,0,0,0,0
0000,1,1,0,0,0,0,0,0,0,0,0
000000,2,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
zshenv,1,0,0,0,0,0,0,0,0,0,0
zubats,1,0,1,0,0,0,0,0,0,0,0
zune,1,0,0,0,1,0,0,0,0,0,0
zypper,1,0,0,0,0,0,0,0,0,0,0


In [109]:
word_counts.sort_values(by='all_words', ascending=False).head(20)

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
poker,1163,208,148,42,86,51,59,45,30,48,0
player,1032,177,162,41,58,78,37,19,4,29,0
game,1013,167,145,58,65,12,15,18,17,30,1
hand,1007,222,71,86,83,38,36,82,16,21,0
card,989,259,84,71,46,23,14,40,23,50,1
run,662,201,52,42,27,43,29,26,1,5,0
1,547,152,42,39,34,25,17,13,9,37,0
use,447,88,63,41,8,25,18,15,5,6,0
code,416,113,47,19,17,9,31,14,6,8,3
install,403,100,12,24,10,27,30,9,4,0,0


In [110]:
word_counts.sort_values(by='all_words', ascending=False).tail(20)

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
marina,1,0,0,0,0,0,0,0,0,0,0
manycore,1,0,0,0,0,0,0,0,0,0,0
mapsmapsreadmemd,1,0,0,0,0,0,0,0,0,0,1
contributorsshield,1,0,0,0,0,1,0,0,0,0,0
mapdocsmapreadmemd,1,0,0,0,0,0,0,0,0,0,1
maphttpsgithubcomrangi42polishedmaptreeplusplus,1,0,0,1,0,0,0,0,0,0,0
contributorshttpsgithubcompropanekplanningpokercontributors,1,1,0,0,0,0,0,0,0,0,0
contributorshttpsgithubcomifs49fpokergraphscontributors,1,1,0,0,0,0,0,0,0,0,0
mapping_,1,0,0,0,0,0,0,0,0,0,0
mapr,1,1,0,0,0,0,0,0,0,0,0


In [111]:
pd.concat([word_counts[word_counts.JavaScript == 0].sort_values(by='JavaScript').tail(6),
           word_counts[word_counts.Java == 0].sort_values(by='Java').tail(6),
           word_counts[word_counts.Cplusplus == 0].sort_values(by='Cplusplus').tail(6),
           word_counts[word_counts.C_sharp == 0].sort_values(by='C_sharp').tail(6),
           word_counts[word_counts.TypeScript == 0].sort_values(by='TypeScript').tail(6),
           word_counts[word_counts.Ruby == 0].sort_values(by='Ruby').tail(6),
           word_counts[word_counts.Cee == 0].sort_values(by='Cee').tail(6),
           word_counts[word_counts.PeHPe == 0].sort_values(by='PeHPe').tail(6),
           word_counts[word_counts.Jupyter_Notebook == 0].sort_values(by='Jupyter_Notebook').tail(6),
           word_counts[word_counts.Assembly == 0].sort_values(by='Assembly').tail(6)])

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
fantasy,2,0,2,0,0,0,0,0,0,0,0
faq,3,0,0,1,0,0,1,0,0,0,0
farger,1,0,0,0,1,0,0,0,0,0,0
farscreenshots2pngrawtrue,1,0,0,0,0,1,0,0,0,0,0
faster,32,0,3,7,3,0,0,0,0,0,0
zzmj,1,0,0,0,0,0,0,0,0,0,0
exp1921680219000,1,1,0,0,0,0,0,0,0,0,0
exp_epsilon,2,0,0,0,0,0,0,0,0,0,0
exp_namenorm_max,1,0,0,1,0,0,0,0,0,0,0
expanded,2,0,0,1,0,0,0,0,0,0,0


In [112]:
word_counts.head(5)

,all_words,JavaScript,Java,Cplusplus,C_sharp,TypeScript,Ruby,Cee,PeHPe,Jupyter_Notebook,Assembly
0,229,79,7,23,7,3,2,1,3,28,0
00,7,0,0,0,0,0,0,0,0,7,0
000,2,0,0,0,0,1,0,0,0,0,0
0000,1,1,0,0,0,0,0,0,0,0,0
000000,2,0,0,0,1,1,0,0,0,0,0


In [113]:
top_20_ham_bigrams = (pd.Series(nltk.ngrams(all_words, 2))
                      .value_counts()
                      .head(20))

top_20_ham_bigrams.head(10)

(texas, holdem)      150
(poker, hand)        123
(planning, poker)    117
(poker, game)         96
(npm, install)        78
(make, sure)          75
(pull, request)       71
(npm, run)            55
(holdem, poker)       53
(source, code)        50
dtype: int64

In [114]:
rooti_tooti = (pd.Series(nltk.ngrams(JavaScript, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(npm, install)       47
(npm, run)           39
(width60, img)       39
(pull, request)      30
(planning, poker)    30
dtype: int64

In [115]:
rooti_tooti = (pd.Series(nltk.ngrams(Java, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(poker, game)      23
(texas, holdem)    22
(unique, id)       21
(de, la)           14
(poker, server)    13
dtype: int64

In [116]:
rooti_tooti = (pd.Series(nltk.ngrams(Cplusplus, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(card, abstraction)       18
(average, strategy)       15
(poker, hand)             11
(betting, abstraction)    10
(monte, carlo)             9
dtype: int64

In [117]:
rooti_tooti = (pd.Series(nltk.ngrams(C_sharp, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(planning, poker)    31
(window, phone)      15
(visual, studio)     13
(small, blind)       11
(texas, holdem)       9
dtype: int64

In [118]:
rooti_tooti = (pd.Series(nltk.ngrams(TypeScript, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(planning, poker)      17
(return, undefined)    15
(property, return)     15
(npm, install)         13
(otherwise, return)    13
dtype: int64

In [119]:
rooti_tooti = (pd.Series(nltk.ngrams(Ruby, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(wed, feb)         16
(0800, 2012)       16
(feb, 01)          16
(poker, hand)      10
(used, project)     9
dtype: int64

In [120]:
rooti_tooti = (pd.Series(nltk.ngrams(Cee, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(poker, hand)             19
(docker, image)           12
(___, ___)                 9
(disassembly, pokemon)     9
(port, used)               7
dtype: int64

In [121]:
rooti_tooti = (pd.Series(nltk.ngrams(PeeHpee, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(x, x)             9
(texas, holdem)    7
(null, null)       7
(3, 3)             7
(k, k)             5
dtype: int64

In [122]:
rooti_tooti = (pd.Series(nltk.ngrams(Java, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(poker, game)      23
(texas, holdem)    22
(unique, id)       21
(de, la)           14
(poker, server)    13
dtype: int64

In [123]:
rooti_tooti = (pd.Series(nltk.ngrams(Jupyter_Notebook, 2))
                      .value_counts()
                      .head(20))
rooti_tooti.head(5)

(0, 1)                      10
(qubit, state)               9
(calculate, probability)     8
(jupyter, notebook)          8
(poker, odds)                8
dtype: int64

Here is a huge win from here David will take the stopwords out and get this project going as for me I am going to need to repeat this process over for all of the rest of the corpus text

Snippets from the explore chapter both in the circulumn and excercise review

Ryan Mcall referenced his <a href="https://github.com/CodeupClassroom/jemison-nlp-exercises/blob/main/prepare_exercises.ipynb">exercise </a>

From here the problem is that if the words are split up what ways do we have to organize them? 

I feel like we can cross that bridge when we get there.

We might need to implement stopwords to get rid of non-useful words.
- the concern here is that words like "the, a, an" are gonna come up as the most often used
    - these do not yeild useful information
    - "tf and idf will account for the stopwords"